https://huggingface.co/transformers/custom_datasets.html#qa-squad

In [27]:
import orjson
import numpy as np
import math

In [3]:
with open("../resources/data_final.json", "rb") as f:
    dataset = orjson.loads(f.read())

In [28]:
np.random.seed(0)
id_train = np.random.choice(len(dataset), math.ceil(len(dataset)/2))

In [30]:
id_train.shape

(48,)

In [29]:
id_test = np.asarray(list(set(range(len(dataset))).difference(set(id_train))))

In [31]:
id_test.shape

(60,)

In [ ]:
from transformers import pipeline
qapipe = pipeline('question-answering')
qapipe({
 'question': """how can question answering service produce answers""",
 'context': """One such task is reading comprehension. Given a passage of text, we can ask questions about the passage that can be answered by referencing short excerpts from the text. For instance, if we were to ask about this paragraph, "how can a question be answered in a reading comprehension task" ..."""
})